In [10]:
import pandas as pd
import os
import json

metric_results = {}
llm_type = 'gpt'
for dataset_name in ['en', 'jp',  'eu']:
    for enhanced in ['enhanced']:
        df = pd.read_csv(f'../step5-close_llm_baseline/results/{dataset_name}/{dataset_name}-{llm_type}.csv')
        df[f'{llm_type}_task'] = df[f'{llm_type}_task'].apply(eval)
        df[f'bert_task'] = df[f'bert_task'].apply(eval)
        df['tag'] = 1
        if enhanced == 'enhanced':
            origin_df = pd.read_excel(f'../step4-llm_select/results/{dataset_name}/{dataset_name}_results.xlsx').drop(['task_top10', 'task'], axis=1)
            origin_df = origin_df[(origin_df['sim']>0.96) & (origin_df['score']==50)]

            filter_df = pd.read_csv(f'../step4-llm_select/results/{dataset_name}/{dataset_name}_filter.csv').drop(['Unnamed: 0'], axis=1)
            filter_df[f'llm_task'] = filter_df[f'llm_task'].apply(eval)
            filter_df[f'bert_task'] = filter_df[f'bert_task'].apply(eval)
            origin_df = pd.merge(origin_df, filter_df, on=['job_title', 'job_description'])
            origin_df['tag'] = 0
            origin_df.columns = ['job_title', 'job_description', 'sim', 'score', 'task', 'gpt_task', 'bert_task', 'tag']
            origin_df = origin_df[df.columns]
            df = pd.concat([df, origin_df]).reset_index(drop=True)
            dataset_name = f"{dataset_name}-enhanced"
        if not os.path.exists(f'data/{dataset_name}'):
            os.makedirs(f'data/{dataset_name}')
        df.to_csv(f'data/{dataset_name}/{dataset_name}-{llm_type}.csv', index=None)
        ans = []
        for i in range(len(df)):
            temp = {'text': df['job_title'][i] + ':' + df['job_description'][i], 'label': df['task'][i]}
            for idx, j in enumerate(df['bert_task'][i]):
                temp[f"top{idx+1}"] = j
            ans.append(temp)
        json.dump(ans, open(f'data/{dataset_name}/rank_bert.json', 'w'))
df

,job_title,job_description,task,gpt_task,bert_task,tag
0,Brand Marketing Manager,Develop and implement marketing strategy for p...,Develop marketing plans or strategies.,"[Develop marketing plans or strategies., Analy...","[Analyze data to inform personnel decisions., ...",1
1,Product Manager,Develop the annual marketing investment plans ...,Develop marketing plans or strategies.,[Develop marketing plans or strategies for env...,[Develop content for sales presentations or ot...,1
2,Director Business Development,* Work with Marketing to develop and implement...,Develop marketing plans or strategies.,"[Develop marketing plans or strategies., Coord...","[Develop marketing plans or strategies., Direc...",1
3,Senior Brand Manager,Developing and leading the brand and marketing...,Develop marketing plans or strategies.,"[Develop marketing plans or strategies., Devel...","[Develop marketing plans or strategies., Direc...",1
4,Advanced Surgery Account Manager,Collaborate closely with Marketing on how to b...,Develop marketing plans or strategies.,"[Develop marketing plans or strategies., Coord...","[Inform the public about policies, services or...",1
...,...,...,...,...,...,...
278,Contract Clinical Study Lead,* Deep understanding of clinical development w...,Conduct research to increase knowledge about m...,[Conduct research to increase knowledge about ...,"[Treat medical emergencies., Supervise inmate ...",0
279,Director of Clinical Development,* Translate findings from research and nonclin...,Conduct research to increase knowledge about m...,[Conduct research to increase knowledge about ...,[Administer compensation or benefits programs....,0
280,Research Physician,Helps explore and evaluate new compound/study ...,Conduct research to increase knowledge about m...,[Conduct research to increase knowledge about ...,[Treat patients using alternative medical proc...,0
281,Territory Manager,Responsible for training other Sales Represent...,Instruct staff in work policies or procedures.,[Instruct staff in work policies or procedures...,[Treat patients using alternative medical proc...,0
